In [114]:
import os 
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im

from sklearn.model_selection import train_test_split


%matplotlib inline 

gear_dir = "gear_images/gear_images"

imageList = []; 
for folder in sorted(os.listdir(gear_dir)):
    folderPath = os.path.join(gear_dir, folder)
    for file in sorted(os.listdir(folderPath)):
        totalPath = os.path.join(gear_dir, folder, file)
        item = (file, totalPath, np.array(Image.open(totalPath)), folder)
        imageList.append(item)

In [115]:
aCoupleImages = imageList
#aCoupleImages = imageList
#print(aCoupleImages)

aCoupleImages[0][3]
labels = []
images = []
for img in aCoupleImages:
    images.append(img[2])
    labels.append(img[3])

def preprocess_images(image_array, target_size=(128, 128)):
    shmimg = []
    
    for img in image_array:
        # Get the original image
        orig_img = Image.fromarray(img) # Get the np.array out of the imgData
        
        scaled_img = orig_img
        scaled_img.thumbnail(target_size, Image.ANTIALIAS)
        scaled_height, scaled_width = scaled_img.size
        new_img = Image.new("RGB", target_size, (255, 255, 255))
        # Paste our new image with the whites removed 
        new_img.paste(scaled_img, (int((target_size[0] - scaled_img.size[0]) / 2), int((target_size[1] - scaled_img.size[1]) / 2)))
        
        img = new_img        
        # img = ImageOps.equalize(Image.fromarray(img))
        shmimg.append(np.array(img).ravel())
    return shmimg

features = np.array(preprocess_images(images))

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.10)
        

In [116]:
from sklearn.linear_model import LogisticRegression

reg = 0.01 

clf = LogisticRegression(C=1/reg, solver='lbfgs', multi_class='multinomial').fit(X_train, Y_train)

print(clf)

LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)


In [117]:
from sklearn import metrics
from sklearn.metrics import accuracy_score 

predictions = clf.predict(X_test)
print('Accuracy: ', accuracy_score(Y_test, predictions))

Accuracy:  0.8169014084507042


In [118]:
testImageUrl="https://shop.epictv.com/sites/default/files/ae42ad29e70ba8ce6b67d3bdb6ab5c6e.jpeg"

import requests
from io import BytesIO

# Takes np array and returns np array resized 
def resizeImage(imageToBeResized, target_size=(128, 128)):
    orig_img = Image.fromarray(imageToBeResized) # Get the np.array out of the imgData
        
    scaled_img = orig_img
    scaled_img.thumbnail(target_size, Image.ANTIALIAS)
    scaled_height, scaled_width = scaled_img.size
    new_img = Image.new("RGB", target_size, (255, 255, 255))
    # Paste our new image with the whites removed 
    new_img.paste(scaled_img, (int((target_size[0] - scaled_img.size[0]) / 2), int((target_size[1] - scaled_img.size[1]) / 2)))
    
    return np.array(new_img)

response = requests.get(testImageUrl)
helmetTestImg = Image.open(BytesIO(response.content))
helmetTestImgNP = np.array(helmetTestImg)
print(helmetTestImgNP.shape)

resized = resizeImage(helmetTestImgNP)

print(resized.shape)


resized?
pred = clf.predict(resized.ravel().reshape(1, -1))
print('The image is a', pred[0])

(1200, 1200, 3)
(128, 128, 3)
The image is a helmets
